In [ ]:
import sys
import pandas as pd
import numpy as np
from math import sqrt
import lmfit
from IPython.display import display, HTML
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
%matplotlib inline
import matplotlib.ticker as ticker
import matplotlib.colors as mcolors
from matplotlib.collections import LineCollection

sys.path.append("../velocity")
import velocity
from velocity.chemistry import Species, Reaction, Network


## Misc formatting and helper functions

In [ ]:
hcolors = {
"red": "#E63745",
"lred": "#F18F99",
"blue": "#1F58B0",
"lblue": "#A8CAF7",
"grey": "#939393",
"lgrey": "#D9D9D9",
"mgrey": "#B8B8B8",
}

font = {
        'weight' : 'bold',
        'size'   : 7,
        'family' : 'sans-serif',}

matplotlib.rc('font', **font)

def convert(selectivity):
    """Converts negative selectivites into inverse selectivities.

    Leaves positive selectivities untouched.

    Args:
        selectivity (int or float): The selectivity to convert.

    Returns:
        selectivity (float): The converted selectivity.
    """
    selectivity = float(selectivity)
    if selectivity < 0.0:
        selectivity = -1.0/selectivity
    return selectivity

def thermo_axis(num):
    """ Helper for formatting thermo axis labels in graphs with thermodynamics
    
    Args:
        num (float): The number to format
        
    Returns:
        str: The formatted axis label
"""
    if num == 6e7:
        return "Thermo"
    else:
        return f"$10^{int(np.log10(num))}$"
    
fmt = ticker.FuncFormatter(lambda x, _: thermo_axis(x))

## Velocity setup

In [ ]:

# parameter bounds
min_log10_base_rate_constant = -8
max_log10_base_rate_constant = 1
min_log10_selectivity = -2
max_log10_selectivity = 2
min_log10_catalyst_deactivation_rate_constant = -6
default_log10_catalyst_deactivation_rate_constant = np.log10(0.000014)
max_log10_catalyst_deactivation_rate_constant = -4

# generate times to run simulation over
t_span = (0, 3600000)
t_eval = [*np.linspace(60*6, 3600000, 1000001)]

# setup sugars
glucose = Species("Glc", "glucose")
mannose = Species("Man", "mannose")
altrose = Species("Alt", "altrose")
allose = Species("All", "allose")
gulose = Species("Gul", "gulose")
galactose = Species("Gal", "galactose")
talose = Species("Tal", "talose")
idose = Species("Ido", "idose")

# must match order in spreadsheet!
sugars = [ glucose, mannose, allose, galactose, altrose, talose, gulose, idose ]
sugars_dict = { s.abbreviation : s for s in sugars }
sugar_abbreviations = [ s.abbreviation for s in sugars ]

# setup catalyst
catalyst_active = Species("cat", "catalyst (active)")
catalyst_dead = Species("dead", "catalyst (dead)")

# these are the network connections
CONNECTIONS = ["Glc_All",  "Glc_Gal",  "Glc_Man",
               "All_Gul",  "Gul_Gal",  "Gal_Tal",
               "Tal_Man",  "Man_Alt",  "Alt_All",
               "Alt_Ido",  "Tal_Ido",  "Ido_Gul"]

def add_connection(reactions_dict, connection_string, base_rate_constant, selectivity):
    sugar1, sugar2 = connection_string.split("_")
    sugar1 = sugars_dict[sugar1]
    sugar2 = sugars_dict[sugar2]

    jk1 = base_rate_constant*sqrt(selectivity)
    jk2 = base_rate_constant/sqrt(selectivity)
    reaction = Reaction({sugar1:1, catalyst_active:1}, {sugar2:1, catalyst_active:1}, reversible=True)
    reactions_dict[reaction] = (jk1,jk2)

def create_network(x, cat_deact=True):
    reactions_dict = {}

    for connection in CONNECTIONS:
        base_rate_constant = np.power(10, x[f"{connection}_log10_base_rate_constant"])
        selectivity = np.power(10, x[f"{connection}_log10_selectivity"])
        add_connection(reactions_dict, connection, base_rate_constant, selectivity)
    if cat_deact:
        catalyst_deactivation_reaction = Reaction(catalyst_active, catalyst_dead, reversible=False)
        reactions_dict[catalyst_deactivation_reaction] = np.power(10.0,x["log10_catalyst_deactivation_rate_constant"])
    
    network = Network(reactions_dict, fixed_concentrations=None)
    return network

def add_parameters(parameters,connection, base_rate_constant, selectivity, vary=(False,False)):
    vary_rate, vary_selectivity = vary
    parameter = lmfit.Parameter(
        name = f"{connection}_log10_base_rate_constant",
        min = min_log10_base_rate_constant,
        value = np.log10(base_rate_constant*1e-6),
        max = max_log10_base_rate_constant,
        vary = True
    )
    parameters.add(parameter)

    parameter = lmfit.Parameter(
        name = f"{connection}_log10_selectivity",
        min = min_log10_selectivity,
        value = np.log10(convert(selectivity)),
        max = max_log10_selectivity,
        vary = False
    )
    parameters.add(parameter)



In [ ]:
def plot_sugar_timecourse(start_sugar,parameters, file=None, plot_title=None, size=(3, 1.5),thermo=False,shading=False,catdeath=True, sugar = "alpha", sugars = ["All","Alt","Gal","Glc","Gul","Ido","Man","Tal"],sugars_catdeath = ["All","Tal"], transparent=False, vertical_time = None, manual_patches = None):
    """
    Plots the simulation results for sugar concentrations over time.

    Parameters:
    - start_sugar (Species): The starting sugar for the simulation.
    - file (str): The file path to save the plot.
    - parameters (lmfit.Parameters): The parameters for the simulation.
    - plot_title (str, optional): The title of the plot. Defaults to None.
    - size (tuple, optional): The size of the plot. Defaults to (3, 1.5).
    - thermo (bool, optional): Whether to include thermodynamic information in the plot. Defaults to False.
    - shading (bool, optional): Whether to include shading in the plot. Defaults to False.
    - catdeath (bool, optional): Whether to include catalyst death in the plot. Defaults to True.
    - sugar (str, optional): The type of sugar to plot thermodynamic data for. Defaults to "alpha".
    - sugars (list, optional): The list of sugars to include in the plot. Defaults to ["All","Alt","Gal","Glc","Gul","Ido","Man","Tal"].
    - sugars_catdeath (list, optional): The list of sugars to include in the plot with catalyst death. Defaults to ["All","Tal"].
    - transparent (bool, optional): Whether to make the plot background transparent. Defaults to False.
    """
    
    #setup colors for sugars:
    colors_dark = {
        "Alt" : hcolors["lgrey"],
        "Gal" : hcolors["lgrey"],
        "Glc" : hcolors["lgrey"],
        "Gul" : hcolors["lgrey"],
        "Ido" : hcolors["lgrey"],
        "Man" : hcolors["lgrey"],
    }
    colors_dark[start_sugar.abbreviation] = hcolors["grey"]
    colors_dark["Tal"] = hcolors["red"]
    colors_dark["All"] = hcolors["blue"]

    colors_light = {
        "All" : hcolors["lblue"],
        "Alt" : hcolors["lgrey"],
        "Gal" : hcolors["lgrey"],
        "Glc" : hcolors["lgrey"],
        "Gul" : hcolors["lgrey"],
        "Ido" : hcolors["lgrey"],
        "Man" : hcolors["lgrey"],
        "Tal" : hcolors["lred"]
    }

    #setup thermodynamic data for sugars:
    if sugar == "alpha":
        sugar_bw = {
            "All" : 0.288778,
            "Alt" : 0.020064,
            "Gal" : 0.147015,
            "Glc" : 0.213119+.01, # shift to make it visible
            "Gul" : 0.035019,
            "Ido" : 0.024568,
            "Man" : 0.065391,
            "Tal" : 0.206045
        }
    elif sugar == "beta":
        sugar_bw = {
            "All" : 0.197289+.01, # shift to make it visible
            "Alt" : 0.023524,
            "Gal" : 0.190740,
            "Glc" : 0.267328,
            "Gul" : 0.033531,
            "Ido" : 0.015426,
            "Man" : 0.087753,
            "Tal" : 0.184409-.01 # shift to make it visible
        }
    else:
        thermo = False

    # setup velocity network initial conditions
    initial_concentrations_dict = {
        start_sugar : 0.2,
        catalyst_active : 0.2*0.02,
    }

    # run the simulation without catalyst death
    network = create_network(parameters,False)
    concentrations_df = network.simulate_timecourse(initial_concentrations_dict, t_span, t_eval)
    concentrations_df = concentrations_df/0.2

    # run the simulation with catalyst death if needed
    if catdeath:
        network_deact = create_network(parameters,True)
        concentrations_df_deact = network_deact.simulate_timecourse(initial_concentrations_dict, t_span, t_eval)
        concentrations_df_deact = concentrations_df_deact/0.2

    # setup subplots
    if thermo:
        fig, axs = plt.subplots(nrows=1, ncols=2, sharey=True, width_ratios=[5, 1], sharex=False,
                                        figsize=size)
        ax1,ax2 = axs
    else:
        fig,ax1 = plt.subplots(figsize=size)
        axs = [ax1]

    # format axes colors and ticks
    for ax in axs:
        ax.xaxis.label.set_color('#939393')        #setting up X-axis label color to yellow
        ax.yaxis.label.set_color('#939393')          #setting up Y-axis label color to blue

        ax.tick_params(axis='x', colors='#939393',which='both')    #setting up X-axis tick color to red
        ax.tick_params(axis='y', colors='#939393',which='both')  #setting up Y-axis tick color to black

        ax.spines['left'].set_color('#939393')        # setting up Y-axis tick color to red
        ax.spines['top'].set_color('#939393')         #setting up above X-axis tick color to red
        ax.spines['bottom'].set_color('#939393')        # setting up Y-axis tick color to red
        ax.spines['right'].set_color('#939393')         #setting up above X-axis tick color to red
        ax.set_yticks([0,0.2,0.4,0.6,0.8,1])
        ax.set_yticklabels(['{:,.0%}'.format(x) for x in ax.get_yticks()])

    # plot catalyst death if needed
    if catdeath:
        for idx, sugar in enumerate(sugars_catdeath,1):
            n = 10
            ax1.plot(concentrations_df_deact.index/60/60, concentrations_df_deact[sugar],'--', color=colors_light[sugar],zorder=n+idx)

    # plot sugar concentrations for each required sugar
    for idx, sugar in enumerate(sugars,1):
        n = 1
        if sugar == start_sugar.abbreviation:
            n = 100
        if (sugar == "Tal") or (sugar == "All"):
            n = 1000

        ax1.plot(concentrations_df.index/60/60, concentrations_df[sugar], label=sugar,color=colors_dark[sugar],zorder=idx+n)
    
    ax1.set_xscale("log")
    ax1.xaxis.set_minor_locator(ticker.LogLocator(numticks=999, subs="auto"))
    ax1.set_xticks([1e0,1e1,1e2,1e3,1e4])
    ax1.set_xlim(6/60,1e3)
    ax1.set_ylim(0,1.0)
    if thermo:
        for sugar in sugars:
            ax2.plot([3e7,1e8],[sugar_bw[sugar],sugar_bw[sugar]],color=colors_dark[sugar],zorder=1)
        ax2.get_yaxis().set_visible(False)
        ax2.get_xaxis().set_visible(False)

    if shading:
        if manual_patches != None:
            for patch in manual_patches:
                ax1.add_patch(patch)
        else:
            if concentrations_df["All"].max() >= concentrations_df["Tal"].max():
                max_sugar = "All"
                shade_color = "#dce7f5"
            else:
                max_sugar = "Tal"
                shade_color = "#fce4e7"
            
            selective_region = concentrations_df[concentrations_df[max_sugar] >= 0.7]
            selective_region["time"] = selective_region.index/60/60
            ax1.add_patch(Rectangle((selective_region.time.min(), 0), selective_region.time.max(), 1,color=shade_color,alpha=1,zorder=1))

            concentrations_df["time"] = concentrations_df.index
            end_tal = concentrations_df[concentrations_df["time"] == concentrations_df["time"].max()].Tal.max()
            end_all = concentrations_df[concentrations_df["time"] == concentrations_df["time"].max()].All.max()
            unsel_region = concentrations_df[((concentrations_df["All"] >= end_all-0.1) & (concentrations_df["All"] <= end_all+0.1)) & ((concentrations_df["Tal"] >= end_tal-0.1) & (concentrations_df["Tal"] <= end_tal+0.1))]
            unsel_region["time"] = unsel_region.index/60/60
            ax1.add_patch(Rectangle((unsel_region.time.min(), 0), unsel_region.time.max(), 1,color="#e4e2ee",alpha=1,zorder=1))

    # draw vertical line on ax1 if needed
    if vertical_time != None:
        ax1.axvline(x=vertical_time, color='#939393', linestyle='--', linewidth=1,zorder=1)

    if transparent:
        fig.set_facecolor('none')
        ax1.set_facecolor('none')
        if thermo:
            ax2.set_facecolor('none')

    plt.xlabel("time (h)",fontweight='bold',size=7)

    if plot_title != None:
        plt.title(plot_title,fontweight='bold',size=8,color='#939393')

    fig.tight_layout(pad=0)

    if file != None:
        plt.savefig(file, dpi=800, bbox_inches="tight")
        plt.close()
    else:
        plt.show()



## Kinetic parameters of the network (from fitting)

In [ ]:
alpha_params = [("Glc_All", 8696.595466168013, 19.41682188288102),
    ("Glc_Gal", 2669.8514888247096, -1.0259999159948268),
    ("Glc_Man", 1700.4648769001128, -3.4503655195119873),
    ("All_Gul", 2272.2408829053693, -21.535708164494086),
    ("Gul_Gal", 9879.277501320068, 2.042525719539408),
    ("Gal_Tal", 1921.3481285089338, 21.44995804709182),
    ("Tal_Man", 3435.4755863963533, -21.71316524913186),
    ("Man_Alt", 8167.951419620186, -1.9331817271119984),
    ("Alt_All", 2251.332181358157, 21.505092729010048),
    ("Alt_Ido", 2243.2621555496744, -1.016210302510667),
    ("Tal_Ido", 3312.9198527517797, -21.611499469946136),
    ("Ido_Gul", 999.9938237781865, 10.614793727460544)]

beta_params = [("Glc_All", 15301.874158372651, 1.1046186389372123),
    ("Glc_Gal", 5406.052704709827, 1.1010294423827778),
    ("Glc_Man", 3081.0571102017593, 5.117632219503642),
    ("All_Gul", 4417.41878099147, -14.893229618571524),
    ("Gul_Gal", 7143.924691122882, 20.6151011423547),
    ("Gal_Tal", 8030.790054565239, 21.818684935483084),
    ("Tal_Man", 6800.291414624909, -21.580338807211955),
    ("Man_Alt", 5514.196193944934, -20.598720669500302),
    ("Alt_All", 2085.3079621791885, 14.222839220674421),
    ("Alt_Ido", 999.9940508958343, 7.109655829761684),
    ("Tal_Ido", 2090.6346900714825, -24.033691493134185),
    ("Ido_Gul", 1182.044361016479, 19.561375790185163)]

### simulate timecourse example

change the starting sugar by changing "glucose" to another sugar Species

In [ ]:
parameters = lmfit.Parameters()

for connection, base_rate, sel in alpha_params:
    add_parameters(parameters, connection, base_rate, sel)

parameter = lmfit.Parameter(
    name = f"log10_catalyst_deactivation_rate_constant",
    min = min_log10_catalyst_deactivation_rate_constant,
    value = default_log10_catalyst_deactivation_rate_constant,
    max = max_log10_catalyst_deactivation_rate_constant,
    vary = False
)
parameters.add(parameter)

plot_sugar_timecourse(glucose,parameters, file=None,size=(3.5, 1.5),catdeath=False,thermo=False,shading=False, transparent=False)

In [ ]:
parameters = lmfit.Parameters()

for connection, base_rate, sel in beta_params:
    add_parameters(parameters, connection, base_rate, sel)

parameter = lmfit.Parameter(
    name = f"log10_catalyst_deactivation_rate_constant",
    min = min_log10_catalyst_deactivation_rate_constant,
    value = default_log10_catalyst_deactivation_rate_constant,
    max = max_log10_catalyst_deactivation_rate_constant,
    vary = False
)
parameters.add(parameter)

plot_sugar_timecourse(glucose,parameters, file=None,size=(3.5, 1.5),catdeath=False,thermo=False,shading=False, transparent=False)

### Example Timecourses in the format of Figure 4C

In [ ]:
parameters = lmfit.Parameters()

for connection, base_rate, sel in alpha_params:
    add_parameters(parameters, connection, base_rate, sel)

parameter = lmfit.Parameter(
    name = f"log10_catalyst_deactivation_rate_constant",
    min = min_log10_catalyst_deactivation_rate_constant,
    value = default_log10_catalyst_deactivation_rate_constant,
    max = max_log10_catalyst_deactivation_rate_constant,
    vary = False
)
parameters.add(parameter)

patches = [Rectangle((0, 0), 1.5, 1,color="#F1F1F1",alpha=1,zorder=1),Rectangle((2, 0), 50, 1,color="#dce7f5",alpha=1,zorder=1),Rectangle((70, 0), 1000, 1,color="#e4e2ee",alpha=1,zorder=1)]

plot_sugar_timecourse(glucose,parameters, file=None,size=(3.5, 1.5),catdeath=True,thermo=True,sugar = "alpha",shading=True, transparent=False, vertical_time = 18, manual_patches = patches)

In [ ]:
parameters = lmfit.Parameters()

for connection, base_rate, sel in beta_params:
    add_parameters(parameters, connection, base_rate, sel)

parameter = lmfit.Parameter(
    name = f"log10_catalyst_deactivation_rate_constant",
    min = min_log10_catalyst_deactivation_rate_constant,
    value = default_log10_catalyst_deactivation_rate_constant,
    max = max_log10_catalyst_deactivation_rate_constant,
    vary = False
)
parameters.add(parameter)

patches = [Rectangle((60, 0), 1000, 1,color="#fce4e7",alpha=1,zorder=1)]

plot_sugar_timecourse(glucose,parameters, file=None,size=(3.5, 1.5),catdeath=True,thermo=True,sugar = "beta",shading=True, transparent=False, vertical_time = 18, manual_patches = patches)

### Timecourses with substitution of beta parameters with alpha ones
SI section 4.10
For Figure 4D

In [ ]:
parameters = lmfit.Parameters()

for connection, base_rate, sel in alpha_params:
    add_parameters(parameters, connection, base_rate, sel)

parameter = lmfit.Parameter(
    name = f"log10_catalyst_deactivation_rate_constant",
    min = min_log10_catalyst_deactivation_rate_constant,
    value = default_log10_catalyst_deactivation_rate_constant,
    max = max_log10_catalyst_deactivation_rate_constant,
    vary = False
)
parameters.add(parameter)

plot_sugar_timecourse(glucose,parameters, file=f"si_mixed_purealpha.png",size=(3.5, 1.5),catdeath=False,thermo=False,shading=False, transparent=False)

In [ ]:
parameters = lmfit.Parameters()

for connection, base_rate, sel in beta_params:
    add_parameters(parameters, connection, base_rate, sel)

parameter = lmfit.Parameter(
    name = f"log10_catalyst_deactivation_rate_constant",
    min = min_log10_catalyst_deactivation_rate_constant,
    value = default_log10_catalyst_deactivation_rate_constant,
    max = max_log10_catalyst_deactivation_rate_constant,
    vary = False
)
parameters.add(parameter)

plot_sugar_timecourse(glucose,parameters,f"si_mixed_purebeta.png",size=(3.5, 1.5),catdeath=False,thermo=False,shading=False, transparent=False)

In [ ]:
parameters = lmfit.Parameters()

for connection, base_rate, sel in beta_params:
    add_parameters(parameters, connection, base_rate, sel)

parameter = lmfit.Parameter(
    name = f"log10_catalyst_deactivation_rate_constant",
    min = min_log10_catalyst_deactivation_rate_constant,
    value = default_log10_catalyst_deactivation_rate_constant,
    max = max_log10_catalyst_deactivation_rate_constant,
    vary = False
)
parameters.add(parameter)

for connection, base_rate, sel in alpha_params:
    local_params = parameters.copy()
    add_parameters(local_params, connection, base_rate, sel)
    plot_sugar_timecourse(glucose,local_params,f"si_mixed_beta_mod_{connection}.png",size=(3.5, 1.5),catdeath=False,thermo=False,shading=False, transparent=False)

## Generate graph of all the timecourses together instead

In [ ]:
def plot_sugar(start_sugar,plotting_sugar,parameters, plot_fig_ax = None,  size=(3, 1.5),transparent=False,alpha=1.0, linewidth=1.5, linestyle="-"):
    colors_dark = {
        "Alt" : hcolors["lgrey"],
        "Gal" : hcolors["lgrey"],
        "Glc" : hcolors["lgrey"],
        "Gul" : hcolors["lgrey"],
        "Ido" : hcolors["lgrey"],
        "Man" : hcolors["lgrey"],
    }
    colors_dark[start_sugar.abbreviation] = hcolors["grey"]
    colors_dark["Tal"] = hcolors["red"]
    colors_dark["All"] = hcolors["blue"]

    colors_light = {
        "All" : hcolors["lblue"],
        "Alt" : hcolors["lgrey"],
        "Gal" : hcolors["lgrey"],
        "Glc" : hcolors["lgrey"],
        "Gul" : hcolors["lgrey"],
        "Ido" : hcolors["lgrey"],
        "Man" : hcolors["lgrey"],
        "Tal" : hcolors["lred"]
    }

    network = create_network(parameters,False)

    initial_concentrations_dict = {
        start_sugar : 0.2,
        catalyst_active : 0.2*0.02,
    }

    concentrations_df = network.simulate_timecourse(initial_concentrations_dict, t_span, t_eval)
    concentrations_df = concentrations_df/0.2

    network_deact = create_network(parameters,True)
    concentrations_df_deact = network_deact.simulate_timecourse(initial_concentrations_dict, t_span, t_eval)
    concentrations_df_deact = concentrations_df_deact/0.2
    font = {
            'weight' : 'bold',
            'size'   : 7,
            'family' : 'sans-serif',}

    matplotlib.rc('font', **font)
    #matplotlib.rc('mathtext.default', **font)

    if plot_fig_ax == None:
        fig,ax = plt.subplots(figsize=size)
    else:
        fig,ax = plot_fig_ax

    ax.xaxis.label.set_color('#939393')        #setting up X-axis label color to yellow
    ax.yaxis.label.set_color('#939393')          #setting up Y-axis label color to blue

    ax.tick_params(axis='x', colors='#939393',which='both')    #setting up X-axis tick color to red
    ax.tick_params(axis='y', colors='#939393',which='both')  #setting up Y-axis tick color to black

    ax.spines['left'].set_color('#939393')        # setting up Y-axis tick color to red
    ax.spines['top'].set_color('#939393')         #setting up above X-axis tick color to red
    ax.spines['bottom'].set_color('#939393')        # setting up Y-axis tick color to red
    ax.spines['right'].set_color('#939393')         #setting up above X-axis tick color to red
    ax.set_yticks([0,0.2,0.4,0.6,0.8,1])
    ax.set_yticklabels(['{:,.0%}'.format(x) for x in ax.get_yticks()])

    ax.plot(concentrations_df_deact.index/60/60, concentrations_df[plotting_sugar.abbreviation], label=plotting_sugar.abbreviation,color=colors_dark[plotting_sugar.abbreviation], alpha=alpha, linestyle=linestyle, linewidth=linewidth)
    
    ax.set_xscale("log")
    ax.xaxis.set_minor_locator(ticker.LogLocator(numticks=999, subs="auto"))
    ax.set_xticks([1e0,1e1,1e2,1e3,1e4])
    ax.set_xlim(6/60,1e3)
    ax.set_ylim(0,1.0)

    if transparent:
        fig.set_facecolor('none')
        ax.set_facecolor('none')

    plt.xlabel("time (h)",fontweight='bold',size=7)

    fig.tight_layout(pad=0)
    

# Generate plots of single substitutions for Figure 4D

In [ ]:
local_fig, local_ax = plt.subplots(figsize=(3,1.5))

parameters = lmfit.Parameters()

for connection, base_rate, sel in beta_params:
    add_parameters(parameters, connection, base_rate, sel)

parameter = lmfit.Parameter(
    name = f"log10_catalyst_deactivation_rate_constant",
    min = min_log10_catalyst_deactivation_rate_constant,
    value = default_log10_catalyst_deactivation_rate_constant,
    max = max_log10_catalyst_deactivation_rate_constant,
    vary = False
)
parameters.add(parameter)
plot_sugar(glucose,talose,parameters, plot_fig_ax=(local_fig,local_ax),size=(3, 1.5), transparent=False, alpha=1, linewidth=2, linestyle="--")

for connection, base_rate, sel in alpha_params:
    local_params = parameters.copy()
    add_parameters(local_params, connection, base_rate, sel)
    plot_sugar(glucose,talose,local_params, plot_fig_ax=(local_fig,local_ax),size=(3, 1.5), transparent=False, alpha=0.5)

plt.savefig("si_mixed_beta_mod_Tal.png", dpi=800, bbox_inches="tight")

In [ ]:
local_fig, local_ax = plt.subplots(figsize=(3,1.5))

parameters = lmfit.Parameters()

for connection, base_rate, sel in beta_params:
    add_parameters(parameters, connection, base_rate, sel)

parameter = lmfit.Parameter(
    name = f"log10_catalyst_deactivation_rate_constant",
    min = min_log10_catalyst_deactivation_rate_constant,
    value = default_log10_catalyst_deactivation_rate_constant,
    max = max_log10_catalyst_deactivation_rate_constant,
    vary = False
)
parameters.add(parameter)
plot_sugar(glucose,allose,parameters, plot_fig_ax=(local_fig,local_ax),size=(3, 1.5), transparent=False, alpha=1, linewidth=2, linestyle="--")

for connection, base_rate, sel in alpha_params:
    local_params = parameters.copy()
    add_parameters(local_params, connection, base_rate, sel)
    plot_sugar(glucose,allose,local_params, plot_fig_ax=(local_fig,local_ax),size=(3, 1.5), transparent=False, alpha=0.5)

plt.savefig("si_mixed_beta_mod_All.png", dpi=800, bbox_inches="tight")

# Generate plots of cumulative substitutions

### Determine steepest descent

In [ ]:
connections = ['Glc_All', 'Gul_Gal', 'Alt_All', 'Glc_Man', 'Man_Alt', 'Glc_Gal', 'All_Gul', 'Alt_Ido', 'Tal_Ido', 'Gal_Tal', 'Tal_Man', 'Ido_Gul']

alpha_dict = {k: (v1, v2) for k, v1, v2 in alpha_params}
beta_dict = {k: (v1, v2) for k, v1, v2 in beta_params}

currendict = beta_dict.copy()
results_cumulative = []

while len(connections) > 0:
    temp_asym_tal = {}
    temp_asym_all = {}    
    temp_dfs = {}
    for connection in connections:
        parameters = lmfit.Parameters()

        add_parameters(parameters, connection, currendict[connection][0],  currendict[connection][1])

        for sugar, conn in currendict.items():
            if sugar != connection:
                add_parameters(parameters, sugar, conn[0],  conn[1])
        
        parameter = lmfit.Parameter(
            name = f"log10_catalyst_deactivation_rate_constant",
            min = min_log10_catalyst_deactivation_rate_constant,
            value = default_log10_catalyst_deactivation_rate_constant,
            max = max_log10_catalyst_deactivation_rate_constant,
            vary = False
        )

        parameters.add(parameter)

        network = create_network(parameters, False)

        initial_concentrations_dict = {
                glucose : 0.2,
                catalyst_active : 0.2*0.02,
        }

        concentrations_df = network.simulate_timecourse(initial_concentrations_dict, t_span, t_eval)
        concentrations_df = concentrations_df*100.0/0.2

        temp_asym_tal[connection] = concentrations_df["Tal"].iloc[-1]
        temp_asym_all[connection] = concentrations_df["All"].iloc[-1]
        temp_dfs[connection] = concentrations_df

    best_connection = min(temp_asym_tal, key=temp_asym_tal.get)

    results_cumulative.append({
        'connection': best_connection,
        'all_asymptote': temp_asym_all[best_connection],
        'tal_asymptote': temp_asym_tal[best_connection],
        'cons_df': temp_dfs[best_connection],
    })

    connections.remove(best_connection)

    currendict[best_connection] = alpha_dict[best_connection]

    print(best_connection)

### So we don't have to run that every time

In [ ]:
cumulative_mods = ["Glc_All",
"Gul_Gal",
"Alt_All",
"Glc_Man",
"Man_Alt",
"Glc_Gal",
"All_Gul",
"Alt_Ido",
"Tal_Ido",
"Gal_Tal",
"Tal_Man",
"Ido_Gul"]

connections = [x[0] for x in alpha_params]

## Generate Cumulative substitution plots
for Figure 4D (right)

In [ ]:
parameters = lmfit.Parameters()
local_fig, local_ax = plt.subplots(figsize=(3,1.5))

for connection, base_rate, sel in beta_params:
    add_parameters(parameters, connection, base_rate, sel)

parameter = lmfit.Parameter(
    name = f"log10_catalyst_deactivation_rate_constant",
    min = min_log10_catalyst_deactivation_rate_constant,
    value = default_log10_catalyst_deactivation_rate_constant,
    max = max_log10_catalyst_deactivation_rate_constant,
    vary = False
)
parameters.add(parameter)
plot_sugar(glucose,allose,parameters, plot_fig_ax=(local_fig,local_ax),size=(3, 1.5), transparent=False, alpha=1, linewidth=2, linestyle="--")

for idx, connection in enumerate(cumulative_mods):
    # get the parameters for the alpha network from the alpha_params list where the connection matches
    base_rate, sel = alpha_params[connections.index(connection)][1:]

    add_parameters(parameters, connection, base_rate, sel)
    plot_sugar(glucose,allose,parameters, plot_fig_ax=(local_fig,local_ax),size=(3, 1.5), transparent=False, alpha=0.5)


In [ ]:
parameters = lmfit.Parameters()
local_fig, local_ax = plt.subplots(figsize=(3,1.5))

for connection, base_rate, sel in beta_params:
    add_parameters(parameters, connection, base_rate, sel)

parameter = lmfit.Parameter(
    name = f"log10_catalyst_deactivation_rate_constant",
    min = min_log10_catalyst_deactivation_rate_constant,
    value = default_log10_catalyst_deactivation_rate_constant,
    max = max_log10_catalyst_deactivation_rate_constant,
    vary = False
)
parameters.add(parameter)
plot_sugar(glucose,talose,parameters, plot_fig_ax=(local_fig,local_ax),size=(3, 1.5), transparent=False, alpha=1, linewidth=2, linestyle="--")

for idx, connection in enumerate(cumulative_mods):
    # get the parameters for the alpha network from the alpha_params list where the connection matches
    base_rate, sel = alpha_params[connections.index(connection)][1:]

    add_parameters(parameters, connection, base_rate, sel)
    plot_sugar(glucose,talose,parameters, plot_fig_ax=(local_fig,local_ax),size=(3, 1.5), transparent=False, alpha=0.5)
